# Extraction of UDS from Neocortex recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd

%matplotlib widget

# add the Contrib dir that contains all tools developped by MB : mbTools.py
#sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'python'))
#print(os.path.join(os.path.dirname(sys.path[0]),'python'))
import mbTools

EMGboolean = pd.read_pickle('EMGframeBoolean_2.pkl')
LFPwakeremoved = np.load('LFPwakeremoved_2.npy', mmap_mode= 'r')
All = np.load('RawDataChannelExtractedDS_2.npy', mmap_mode= 'r')
PFC = All[:, 1]
PFCwakeremoved = LFPwakeremoved[:,0]
S1 = All[:, 2]
S1wakeremoved = LFPwakeremoved[:,1]

# Band pass filter
        Theta-Gamma: 8-50 Hz

In [ ]:
# Filtre parameter:
f_lowcut = 8.
f_hicut = 50.
fs = 1000
nyq = 0.5 * fs
N = 4                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')
filt_PFC = signal.filtfilt(b, a, PFC)
filt_PFCwakeremoved = signal.filtfilt(b, a, PFCwakeremoved)

filt_S1 = signal.filtfilt(b, a, S1)
filt_S1wakeremoved = signal.filtfilt(b, a, S1wakeremoved)

# Plot
# times = np.arange(0, PFC.size/fs, 1./fs)
# timesmin = np.arange(0, PFC.size/fs/60, 1./fs/60)
# fig, ax = plt.subplots()
# ax.plot(timesmin, filt_S1)

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [ ]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(8, 50, 43)
widths = w*fs / (2*freq*np.pi)
PFCNWcwt = signal.cwt(filt_PFCwakeremoved, signal.morlet2, widths, w=w)
S1NWcwt = signal.cwt(filt_S1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation PFC
absPFCNWcwt = np.absolute(PFCNWcwt)
proj_PFCNWcwt = np.sum(absPFCNWcwt, axis = 0)/43
sdproj_PFCcwt = np.std(proj_PFCNWcwt)
sd3proj_PFCcwt = sdproj_PFCcwt*3

# Projection calculation S1
absS1NWcwt = np.absolute(S1NWcwt)
proj_S1NWcwt = np.sum(absS1NWcwt, axis = 0)/43
sdproj_S1cwt = np.std(proj_S1NWcwt)
sd3proj_S1cwt = sdproj_S1cwt*3

PFCNWcwt = None
S1NWcwt = None

freq

Second on the signal for which wake times have been zeroed

DC removed

Before computing the projection, multiply CWT by frequency vector.

Last multiply matrix by inverted signal

In [ ]:
#####################################
########        PFC         #########
#####################################
# DC
mPFCw0L = np.mean(PFCwakeremoved)
mPFCw0C = np.mean(PFCwakeremoved)

# Conservative boolean filtering of PFC filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fPFCwake0C = filt_PFC.copy()
fPFCwake0C[BooleanCons] = 0
PFCwake0C = PFC.copy()
PFCwake0C = PFCwake0C - mPFCw0C
PFCwake0C[BooleanCons] = 0
# Liberal boolean filtering of PFC filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fPFCwake0L = filt_PFC.copy()
fPFCwake0L[BooleanLib] = 0
PFCwake0L = PFC.copy()
PFCwake0L = PFCwake0L - mPFCw0L
PFCwake0L[BooleanLib] = 0

# Computation of CWT
PFCcwtWake0cons = signal.cwt(fPFCwake0C, signal.morlet2, widths, w=w)
PFCcwtWake0lib = signal.cwt(fPFCwake0L, signal.morlet2, widths, w=w)

# Normalisation
PFCcwtWake0consN = np.transpose(np.transpose(PFCcwtWake0cons) * freq)
PFCcwtWake0libN = np.transpose(np.transpose(PFCcwtWake0lib) * freq)

# Projection calculation
absPFCW0Ccwt = np.absolute(PFCcwtWake0consN)
proj_PFCW0Ccwt = np.sum(absPFCW0Ccwt, axis = 0)/1000
absPFCW0Lcwt = np.absolute(PFCcwtWake0libN)
proj_PFCW0Lcwt = np.sum(absPFCW0Lcwt, axis = 0)/1000

# Kill CWT (to save memory)
PFCcwtWake0cons = None
PFCcwtWake0lib = None
PFCcwtWake0consN = None
PFCcwtWake0libN = None

# To input inverted signal into projection
projWsignal_PFCW0C = proj_PFCW0Ccwt * (- PFCwake0C) / 10000
projWsignal_PFCW0L = proj_PFCW0Lcwt * (- PFCwake0L)/ 10000




In [ ]:
#####################################
########         S1         #########
#####################################
# DC
mS1w0L = np.mean(S1wakeremoved)
mS1w0C = np.mean(S1wakeremoved)

# Conservative boolean filtering of S1 filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fS1wake0C = filt_S1.copy()
fS1wake0C[BooleanCons] = 0
S1wake0C = S1.copy()
S1wake0C = S1wake0C - mS1w0C
S1wake0C[BooleanCons] = 0
# Liberal boolean filtering of S1 filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fS1wake0L = filt_S1.copy()
fS1wake0L[BooleanLib] = 0
S1wake0L = S1.copy()
S1wake0L = S1wake0L - mS1w0L
S1wake0L[BooleanLib] = 0

# Computation of CWT
S1cwtWake0cons = signal.cwt(fS1wake0C, signal.morlet2, widths, w=w)
S1cwtWake0lib = signal.cwt(fS1wake0L, signal.morlet2, widths, w=w)

# Normalisation
S1cwtWake0consN = np.transpose(np.transpose(S1cwtWake0cons) * freq)
S1cwtWake0libN = np.transpose(np.transpose(S1cwtWake0lib) * freq)

# Projection calculation
absS1W0Ccwt = np.absolute(S1cwtWake0cons)
proj_S1W0Ccwt = np.sum(absS1W0Ccwt, axis = 0)/1000
absS1W0Lcwt = np.absolute(S1cwtWake0lib)
proj_S1W0Lcwt = np.sum(absS1W0Lcwt, axis = 0)/1000

# Kill CWT (to save memory)
S1cwtWake0cons = None
S1cwtWake0lib = None
S1cwtWake0consN = None
S1cwtWake0libN = None

# To input inverted signal into projection
projWsignal_S1W0C = proj_S1W0Ccwt * (- S1wake0C) / 100
projWsignal_S1W0L = proj_S1W0Lcwt * (- S1wake0L)/ 100

### Display subset 

Not necessary cell to run

In [ ]:
# Defining subset
start = 00000
end = 800000

times = np.arange(0, PFC.size/fs, 1./fs)
tt = times[start:end]
Cortext = PFCwake0L[start:end]/10
Cortext2 = S1wake0L[start:end]/10
proj_Cortexcwtt = proj_PFCW0Lcwt[start:end]

plt.close('all')
plt.axhline(sdproj_PFCcwt, color='r') # horizontal
plt.axhline(sd3proj_PFCcwt, color='g') # horizontal
plt.plot(tt, Cortext2)
plt.plot(tt, projWsignal_S1W0C)
plt.show()



## Extracting Down states and determining main properties 

WORK IN PROGRESS. From Igor, painful. Needs to be done on a nice signal.

In [ ]:
	variable ic5 = 0
	for (ic5=0;ic5<numcols;ic5+=1)
		if(multall[ic5]<0)
			Sums4[ic5]=0
		else
			Sums4[ic5]=1
		endif
	endfor

SetScale /P x, 0, 0.001, Sums4

//// Maintenant on va trier ces donnees pour determiner les bornes des up-states et virer les artefacts
	wave bilan
	make/o/n = (numcols,4) bilan = 0
// Dans cette premiere boucle on repere toutes les transitions 0 -> 1 et 1 -> 0. On prend aussi en compte si la wave commence ou finit par 0 ou 1
	variable ic6 = 0
	variable j6 = 0
		for (ic6=0;ic6<(numcols-1);ic6+=1)
		// les fichiers qui commencent par un DS
			if(Sums4[0]==0)
				if(Sums4[ic6]==0 && Sums4[ic6+1]==1)
					bilan[j6][0]=ic6
				elseif(Sums4[ic6]==1 && Sums4[ic6+1]==0)
					bilan[j6][1]=ic6
					bilan[j6][2]=bilan[j6][1]-bilan[j6][0]
					bilan[j6][3]=bilan[j6][0]-bilan[j6-1][1]
					j6+=1
				endif
		// les fichiers qui commencent par un US
			elseif(Sums4[0]==1)
				bilan[0][0]=0
				if(Sums4[ic6]==0  && Sums4[ic6+1]==1)
					bilan[j6][0]=ic6
				elseif(Sums4[ic6]==1 && Sums4[ic6+1]==0)
					bilan[j6][1]=ic6
					bilan[j6][2]=bilan[j6][1]-bilan[j6][0]
					bilan[j6][3]=bilan[j6][0]-bilan[j6-1][1]
					j6+=1
				endif
			endif
		endfor
		if(Sums4[numcols]==1)
			bilan[j6][1]=numcols
			bilan[j6][2]=bilan[j6][1]-bilan[j6][0]
			bilan[j6][3]=bilan[j6][0]-bilan[j6-1][1]	
		endif
		deletepoints j6, numcols-j6, bilan


	wave bilan
	Variable numRows2 = DimSize(bilan,0)

	Duplicate/o	bilan bilan2



// Et pour les autres, on va accumuler les down-states qui sont separes de moins de 50ms autour d'un Down-state de 100ms.

		bilan2[0][3] = NaN
		variable ic7
// on enleve toutes les petites transitions de moins de 40ms.
		for (ic7=1;ic7<(numRows2);ic7+=1)
			if((bilan2[ic7][2] + bilan2[ic7][3])< 40)// 
				bilan2[ic7-1][1] = bilan2[ic7][1]
				bilan2[ic7-1][2] = bilan2[ic7-1][1] - bilan2[ic7-1][0]
				bilan2[ic7+1][3] = bilan2[ic7+1][0] - bilan2[ic7-1][1]
				deletepoints/M=0 ic7, 1, bilan2
				ic7-=1
			endif
		endfor

// on enleve tous les petits down-states de moins de 100ms
		for (ic7=1;ic7<(numRows2);ic7+=1)
			if(bilan2[ic7][3]< 100)//
				bilan2[ic7-1][1] = bilan2[ic7][1]
				bilan2[ic7-1][2] = bilan2[ic7-1][1] - bilan2[ic7-1][0]
				bilan2[ic7+1][3] = bilan2[ic7+1][0] - bilan2[ic7-1][1]
				deletepoints/M=0 ic7, 1, bilan2
				ic7-=1
			endif
		endfor



////// Finalement on enleve tous les up-states qui font moins de 100ms parce que pareil c'est un artefact.
		for (ic7=1;ic7<(numRows2);ic7+=1)
			if(bilan2[ic7][2]< 50)//
				bilan2[ic7-1][1] = bilan2[ic7][1]
				bilan2[ic7-1][2] = bilan2[ic7-1][1] - bilan2[ic7-1][0]
				bilan2[ic7+1][3] = bilan2[ic7+1][0] - bilan2[ic7-1][1]
				deletepoints/M=0 ic7, 1, bilan2
				ic7-=1
			endif
		endfor


	Variable numRows3 = DimSize(bilan2,0)

// fin : creation du overlay pour une meilleure visualisation
Duplicate/O Sums3, Sums5
variable numcols32 = numpnts(Sums3)
Sums5 = 0
	variable ic9 = 0
	ic2 = 0	
	for (ic9=0;ic9<(numRows3);ic9+=1)
	for(ic2 = bilan2[ic9][0]; ic2 <= bilan2[ic9][1]; ic2 +=1)
		if(bilan2[ic9][2]>2500)
			Sums5[ic2] = 2
		else
			Sums5[ic2] = 1
		endif
	endfor
	endfor
